In [8]:
import psycopg2 as pg
with open('credentials.txt') as creds:
    db,user,host,port = creds.read().splitlines()
    
     
connection = pg.connect(dbname=db, user=user, host=host, port=port)
cursor = connection.cursor()

In [6]:
cursor.execute('create extension cube;')

In [6]:
# группы возрастов
cursor.execute("""CREATE TABLE age_groups (
                age_group_id serial PRIMARY KEY, 
                lower int,
                upper int
               );""")

age_list = [(0,17),(18,24),(25,34),(35,44),(45,54),(55,64),(65,100)]
for age_range in age_list:
    cursor.execute("INSERT INTO age_groups(lower,upper) VALUES(%s,%s)",age_range)


# таблица пользователей
cursor.execute("""CREATE TABLE users (
                user_id serial PRIMARY KEY, 
                vector cube,
                gender bool,
                age_group_id int references age_groups (age_group_id) );""")


# таблица заказов 
cursor.execute("""CREATE TABLE orders (
                order_id serial PRIMARY KEY, 
                user_id int references users (user_id) ON DELETE CASCADE);""")

# таблицa продуктов
cursor.execute("CREATE TABLE items (item_id serial PRIMARY KEY, item_name varchar);")

# m2m-таблица, соотносящая заказы и продукты (состав заказов)
cursor.execute("""CREATE TABLE orders_items (
                order_id int references orders (order_id) ON DELETE CASCADE, 
               item_id int references items (item_id));""")

# таблица тегов
cursor.execute("CREATE TABLE tags (tag_id serial PRIMARY KEY, tag_name varchar);")
    
# m2m-таблица, соотносящая продукты и теги (описание продуктов)
cursor.execute("""create table items_tags (
               item_id int references items (item_id), 
               tag_id int references tags (tag_id));""")

In [7]:
connection.commit()
connection.close()

In [12]:
#connection.rollback()